**Install Requirements**

In [0]:
!pip3 install 'torch==1.4.0'
!pip3 install 'torchvision==0.5.0'
!pip3 install 'Pillow-SIMD'
!pip3 install 'tqdm'

In [0]:
import os
if not os.path.isdir('./Pacs'):
  !git clone https://github.com/lore-lml/machine-learning2020-hw3.git
  !mv 'machine-learning2020-hw3' 'Pacs'
  !rm './Pacs/hw3.ipynb'
  !rm './Pacs/README.md'

import logging

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn

import torchvision
from torchvision import transforms
from Pacs.pacs_dataset import Pacs
from Pacs.dann import alexdann
from Pacs.dann import train_src, test_target, dann_train_src_target

from PIL import Image
from tqdm import tqdm

import matplotlib.pyplot as plt
%matplotlib inline

**Set Arguments**

In [0]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

NUM_CLASSES = 7

BATCH_SIZE = 128     # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                     # the batch size, learning rate should change by the same factor to have comparable results

LR = 1e-2        # The initial Learning Rate
MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-5  # Regularization, you can keep this at the default

NUM_EPOCHS = 10      # Total number of training epochs (iterations over dataset)
STEP_SIZE = 7       # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1          # Multiplicative factor for learning rate step-down

ALPHA = 0.1
BASE_FILE_PATH = "DA_RUN18_LR3e-5_ADAMW_EP15_SS15_G01_ALL_TRANSF"

**Define Data Preprocessing**

In [0]:
transforms = transforms.Compose([transforms.Resize(256),      # Resizes short size of the PIL image to 256
                                      transforms.CenterCrop(224),  # Crops a central square patch of the image
                                                                   # 224 because torchvision's AlexNet needs a 224x224 input!
                                                                   # Remember this when applying different transformations, otherwise you get an error
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)) # Normalizes tensor with mean and standard deviation
])

**Prepare Dataset**

In [4]:
ROOT = 'Pacs/PACS'

source_data = Pacs(ROOT, transform=transforms, source='photo')
target_data = Pacs(ROOT, transform=transforms, source='art_painting')

_, source_labels = source_data.get_img_with_labels()
_, target_labels = target_data.get_img_with_labels()

print(f"# classes source_data: {len(set(source_labels))}")
print(f"# classes val_set: {len(set(target_labels))}")
print(f"source_data: {len(source_data)} elements")
print(f"target_data: {len(target_data)} elements")

# classes source_data: 7
# classes val_set: 7
source_data: 1670 elements
target_data: 2048 elements


**Prepare Dataloaders**

In [0]:
source_dataloader = DataLoader(source_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
target_dataloader = DataLoader(target_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
test_dataloader = DataLoader(target_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, drop_last=False)

**Prepare Network**

In [6]:
def init_cnn_objects(model):
  
  # Define loss function
  criterion = nn.CrossEntropyLoss() # for classification, we use Cross Entropy
  parameters_to_optimize = model.parameters() # In this case we optimize over all the parameters of AlexNet
  
  optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
  #optimizer = optim.Adam(parameters_to_optimize, lr=LR,amsgrad=True)
  #optimizer = optim.AdamW(parameters_to_optimize, lr=LR,amsgrad=True, weight_decay=WEIGHT_DECAY)
  scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

  return criterion, optimizer, scheduler

dann = alexdann(pretrained=True)
criterion, optimizer, scheduler = init_cnn_objects(dann)
print("******* NET CREATED *******")

******* NET CREATED *******


**Training**

In [0]:
def simple_train_test(model, source_dataloader, test_dataloader, file_path=BASE_FILE_PATH):
    train_losses = []
    loss_min = -1
    
    model = model.to(DEVICE)
    cudnn.benchmark
    
    current_step = 0
    for epoch in range(NUM_EPOCHS):
        print('Starting epoch {}/{}, LR = {}'.format(epoch+1, NUM_EPOCHS, scheduler.get_lr()))
        cumulative_loss, current_step = train_src(model, source_dataloader, optimizer, criterion, current_step, DEVICE)
        curr_loss = cumulative_loss / len(source_dataloader)
        train_losses.append(curr_loss)
        if loss_min == -1 or loss_min > curr_loss:
            loss_min = curr_loss
            torch.save(model, f"{file_path}_best_model.pth")
        scheduler.step()
        
    model = torch.load(f"{file_path}_best_model.pth").to(DEVICE)
    accuracy = test_target(model, test_dataloader, criterion, DEVICE) / float(len(target_data))
    print(f"Accuracy on test set: {accuracy}%")
    return train_losses

In [8]:
train_losses = simple_train_test(dann, source_dataloader, test_dataloader)

Starting epoch 1/10, LR = [0.01]
Step 0, Loss_train 2.0163979530334473
Step 10, Loss_train 0.14839710295200348
Starting epoch 2/10, LR = [0.01]
Step 20, Loss_train 0.29159054160118103
Starting epoch 3/10, LR = [0.01]
Step 30, Loss_train 0.1257377415895462
Starting epoch 4/10, LR = [0.01]
Step 40, Loss_train 0.02048277109861374
Step 50, Loss_train 0.07205040007829666
Starting epoch 5/10, LR = [0.01]
Step 60, Loss_train 0.027401000261306763
Starting epoch 6/10, LR = [0.01]
Step 70, Loss_train 0.004358857870101929
Starting epoch 7/10, LR = [0.01]
Step 80, Loss_train 0.016971617937088013
Step 90, Loss_train 0.006642531603574753
Starting epoch 8/10, LR = [0.0001]
Step 100, Loss_train 0.0011034831404685974
Starting epoch 9/10, LR = [0.001]
Step 110, Loss_train 0.0011677518486976624
Starting epoch 10/10, LR = [0.001]
Step 120, Loss_train 0.00018543750047683716


100%|██████████| 16/16 [00:06<00:00,  2.66it/s]

Accuracy on test set: 0.48486328125%


**Training with DANN**

In [0]:
def dann_train_test(model, source_dataloader, target_dataloader, test_dataloader, file_path=BASE_FILE_PATH):
    src_losses_y = []
    src_losses_d = []
    tgt_losses_d = []
    loss_min = -1

    len_dataloader = min(len(source_dataloader), len(target_dataloader))
    
    model = model.to(DEVICE)
    cudnn.benchmark
    
    current_step = 0
    for epoch in range(NUM_EPOCHS):
        print('Starting epoch {}/{}, LR = {}'.format(epoch+1, NUM_EPOCHS, scheduler.get_lr()))
        cum_loss_src_y, cum_loss_src_d, cum_loss_tgt_d, current_step = dann_train_src_target(model, source_dataloader, target_dataloader, optimizer, criterion, current_step, alpha=ALPHA, device=DEVICE)
        curr_loss = cum_loss_src_y / len_dataloader
        src_losses_y.append(curr_loss)
        src_losses_d.append(cum_loss_src_d / len_dataloader)
        tgt_losses_d.append(cum_loss_tgt_d / len_dataloader)
        if loss_min == -1 or loss_min > curr_loss:
            loss_min = curr_loss
            torch.save(model, f"{file_path}_best_model.pth")
        scheduler.step()
        
    model = torch.load(f"{file_path}_best_model_dann.pth").to(DEVICE)
    accuracy = test_target(model, test_dataloader, criterion, DEVICE) / float(len(target_data))
    print(f"Accuracy on test set: {accuracy}%")
    return src_losses_y, src_losses_d, tgt_losses_d

In [0]:
dann = alexdann(pretrained=True)
criterion, optimizer, scheduler = init_cnn_objects(dann)
src_losses_y, src_losses_d, tgt_losses_d = dann_train_test(dann, source_dataloader, target_dataloader, test_dataloader)